# World Bank API Analysis

Pregunta guía: **¿Qué países mejoraron más su esperanza de vida en 2010-2024 y cómo se relaciona con su PIB per cápita?**

## Plan del análisis

- Países: `USA`, `DEU`, `CHN`, `BRA`, `NGA`
- Indicadores: esperanza de vida, PIB per cápita y población
- Rango temporal: 2010 a 2024

In [3]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

sns.set_theme(style="whitegrid")

BASE_URL = "https://api.worldbank.org/v2"
COUNTRIES = ["USA", "DEU", "CHN", "BRA", "NGA"]
YEARS = (2010, 2024)

INDICATORS = {
    "life_expectancy": "SP.DYN.LE00.IN",
    "gdp_per_capita": "NY.GDP.PCAP.CD",
    "population": "SP.POP.TOTL"
}

In [5]:
def fetch_indicator_data(indicator_code, countries, start_year=2010, end_year=2024):
    country_str = ";".join(countries)
    url = f"{BASE_URL}/country/{country_str}/indicator/{indicator_code}"
    per_page = 100
    page = 1
    total_pages = 1
    rows = []

    while page <= total_pages:
        params = {
            "format": "json",
            "per_page": per_page,
            "page": page
        }

        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        payload = response.json()

        if not isinstance(payload, list) or len(payload) < 2 or payload[1] is None:
            break

        metadata = payload[0]
        total_pages = int(metadata.get("pages", 1))

        for item in payload[1]:
            year = int(item["date"])
            if start_year <= year <= end_year:
                rows.append({
                    "country": item["country"]["value"],
                    "countryiso3code": item["countryiso3code"],
                    "year": year,
                    "value": item["value"],
                    "indicator_code": indicator_code
                })

        page += 1

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["value"]).copy()
    df = df.sort_values(["countryiso3code", "year"]).reset_index(drop=True)
    return df

In [ ]:
dfs = {}
for name, code in INDICATORS.items():
    dfs[name] = fetch_indicator_data(code, COUNTRIES, YEARS[0], YEARS[1])
    print(name, dfs[name].shape)

df_life = dfs["life_expectancy"]
df_gdp = dfs["gdp_per_capita"]
df_pop = dfs["population"]

df_life.head()

## Cambio de esperanza de vida (2010-2024)

In [ ]:
life_2010 = (
    df_life[df_life["year"] == 2010][["countryiso3code", "value"]]
    .rename(columns={"value": "life_2010"})
)

life_2024 = (
    df_life[df_life["year"] == 2024][["countryiso3code", "country", "value"]]
    .rename(columns={"value": "life_2024"})
)

life_change = life_2024.merge(life_2010, on="countryiso3code", how="inner")
life_change["life_change_2010_2024"] = life_change["life_2024"] - life_change["life_2010"]
life_change = life_change.sort_values("life_change_2010_2024", ascending=False).reset_index(drop=True)

life_change

## Relación con PIB per cápita (2024)

In [ ]:
gdp_2024 = (
    df_gdp[df_gdp["year"] == 2024][["countryiso3code", "value"]]
    .rename(columns={"value": "gdp_pc_2024"})
)

analysis_df = life_change.merge(gdp_2024, on="countryiso3code", how="left")
analysis_df = analysis_df[["country", "countryiso3code", "life_2010", "life_2024", "life_change_2010_2024", "gdp_pc_2024"]]
analysis_df.sort_values("life_change_2010_2024", ascending=False)

## Visualización 1: evolución de esperanza de vida

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_life, x="year", y="value", hue="countryiso3code", marker="o")
plt.title("Evolución de esperanza de vida (2010-2024)")
plt.xlabel("Año")
plt.ylabel("Esperanza de vida (años)")
plt.legend(title="País")
plt.show()

## Visualización 2: mejora en esperanza de vida vs PIB per cápita

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=analysis_df, x="gdp_pc_2024", y="life_change_2010_2024", hue="countryiso3code", s=120)

for _, row in analysis_df.iterrows():
    plt.text(row["gdp_pc_2024"], row["life_change_2010_2024"], row["countryiso3code"])

plt.title("Mejora de esperanza de vida vs PIB per cápita (2024)")
plt.xlabel("PIB per cápita 2024 (USD actuales)")
plt.ylabel("Cambio en esperanza de vida 2010-2024 (años)")
plt.show()

## Carga a SQLite

In [ ]:
engine = create_engine("sqlite:///world_bank_analysis.db")

df_life.to_sql("indicator_life_expectancy", con=engine, if_exists="replace", index=False)
df_gdp.to_sql("indicator_gdp_per_capita", con=engine, if_exists="replace", index=False)
df_pop.to_sql("indicator_population", con=engine, if_exists="replace", index=False)
analysis_df.to_sql("analysis_life_vs_gdp", con=engine, if_exists="replace", index=False)

pd.read_sql("SELECT * FROM analysis_life_vs_gdp LIMIT 5", con=engine)

## Hallazgos (completar al ejecutar)

1. País con mayor mejora de esperanza de vida: ...
2. Relación observada con PIB per cápita: ...
3. Diferencias destacadas entre países: ...